In [1]:
import pandas as pd
import requests
import json

In [2]:
def req(obj):
    url = 'https://ring.org.ua/search?q=' + obj + '&datasources=edrdr&format=json'
    response = requests.get(url) # надсилаємо get-запит на сервер ring.org.ua
    #response = requests.get(url, verify=False) # якщо матюкається на SSL-сертифікат
    if response.status_code == 200:
        dict_resp = response.json()
        items = dict_resp['search_results']['paginator']['count'] # кількість пов'язаних фірм
        pages = dict_resp['search_results']['paginator']['num_pages'] # кількість сторінок у результаті
        result = dict_resp['search_results']['object_list'] # формуємо список даних про фірми
        if pages > 1: # якщо результати запиту не вмістилися на одну сторінку (не може бути більше 1000?)
            for i in range(2, pages+1): # повторюємо запит для всіх сторінок, починаючи з другої
                url = 'https://ring.org.ua/search?q=' + obj + '&datasources=edrdr&format=json&page=' + str(i)
                response = requests.get(url) # надсилаємо get-запит на сервер ring.org.ua
                #response = requests.get(url, verify=False) # якщо матюкається на SSL-сертифікат
                if response.status_code == 200:
                    dict_resp = response.json()
                    result += dict_resp['search_results']['object_list'] # поповнюємо список даних про фірми
                else:
                    print('Error: requested url ' + url + ' is not available!')
        if len(result) == items: # перевіряємо, чи нам вдалося отримати інформацію про всі пов'язані фірми
            return (items, result)
        else:
            print('Warning: parsed url ' + url.split('&page=')[0] + ' is not full!')
            return (len(result), result)
    else:
        print('Error: requested url ' + url + ' is not available!')
        return (0, ['error'])

In [3]:
def reg(loc):
    if (loc.find('київська область') != -1) or (loc.find('київська обл.') != -1):
        return 1
    elif (loc.find('дніпропетровська область') != -1) or (loc.find('дніпропетровська обл.') != -1):
        return 2
    elif (loc.find('донецька область') != -1) or (loc.find('донецька обл.') != -1):
        return 3
    elif (loc.find('харківська область') != -1) or (loc.find('харківська обл.') != -1):
        return 4
    elif (loc.find('одеська область') != -1) or (loc.find('одеська обл.') != -1):
        return 5
    elif (loc.find('львівська область') != -1) or (loc.find('львівська обл.') != -1):
        return 6
    elif (loc.find('миколаївська область') != -1) or (loc.find('миколаївська обл.') != -1):
        return 7
    elif (loc.find('запорізька область') != -1) or (loc.find('запорізька обл.') != -1):
        return 8
    elif (loc.find('хмельницька область') != -1) or (loc.find('хмельницька обл.') != -1):
        return 9
    elif (loc.find('полтавська область') != -1) or (loc.find('полтавська обл.') != -1):
        return 10
    elif (loc.find('луганська область') != -1) or (loc.find('луганська обл.') != -1):
        return 11
    elif (loc.find('вінницька область') != -1) or (loc.find('вінницька обл.') != -1):
        return 12
    elif (loc.find('житомирська область') != -1) or (loc.find('житомирська обл.') != -1):
        return 13
    elif (loc.find('черкаська область') != -1) or (loc.find('черкаська обл.') != -1):
        return 14
    elif (loc.find('херсонська область') != -1) or (loc.find('херсонська обл.') != -1):
        return 15
    elif (loc.find('кіровоградська область') != -1) or (loc.find('кіровоградська обл.') != -1):
        return 16
    elif (loc.find('сумська область') != -1) or (loc.find('сумська обл.') != -1):
        return 17
    elif (loc.find('івано-франківська область') != -1) or (loc.find('івано-франківська обл.') != -1):
        return 18
    elif (loc.find('тернопільська область') != -1) or (loc.find('тернопільська обл.') != -1):
        return 19
    elif (loc.find('чернігівська область') != -1) or (loc.find('чернігівська обл.') != -1):
        return 20
    elif (loc.find('волинська область') != -1) or (loc.find('волинська обл.') != -1):
        return 21
    elif (loc.find('рівненська область') != -1) or (loc.find('рівненська обл.') != -1):
        return 22
    elif (loc.find('закарпатська область') != -1) or (loc.find('закарпатська обл.') != -1):
        return 23
    elif (loc.find('чернівецька область') != -1) or (loc.find('чернівецька обл.') != -1):
        return 24
    elif (loc.find('республіка крим') != -1) or (loc.find(' арк ') != -1) or (loc.find(' арк,') != -1):
        return 25
    elif (loc.find(' київ ') != -1) or (loc.find(' київ,') != -1):
        return 1
    elif (loc.find(' севастополь ') != -1) or (loc.find(' севастополь,') != -1):
        return 25
    else:
        return 0 # адреса не підходить під жодну з наших масок, тому на неї далі не звертатимемо уваги

In [4]:
def add_info(firm, region=0):
    location = firm['latest_record']['location'].lower().replace('  ', ' ')
    address = reg(location) # знаходимо регіон, де розташована фірма
    if (region == 0) or (region == address): # фірми з іншого регіону не розглядаємо (засновники -- однофамільці?)
        # додаємо фірму до словника знайдених контрагентів
        edrpou = firm['full_edrpou']
        name = firm['latest_record']['name'].lower()
        short_name = firm['latest_record']['short_name'].lower()
        status = firm['latest_record']['status'].lower()
        dict_find[edrpou] = (name, short_name, status) # додаємо інформацію про фірму: назва, повна назва, статус
        dict_firm[name] = edrpou # якщо будуть дублі за повною назвою фірми, то запишеться лише останній ЄДРПОУ!
        if (status != 'припинено') and (name not in list_temp) and (name not in list_next): # якщо фірма функціонує
            list_next.append((name, )) # на наступному рівні шукатимемо засновані цією фірмою фірми
            # додаємо інформацію про засновників фірми
            list_all = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons']])) # фізособи
            for person in list_all:
                if person not in set(dict_find.keys()): # якщо ця особа ще не засвітилася раніше
                    dict_find[person] = () # додаємо її П.І.Б. до словника знайдених контрагентів
                if (person not in list_temp) and (person not in list_next): # і якщо вона не включена до пошуку
                    list_next.append((person, address)) # на наступному рівні шукатимемо засновані цією особою фірми
            list_master = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                    if person[1] == 'Голова'])) # список голів фірми (голова тільки один?)
            list_founder = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                     if person[1] == 'Засновник'])) # список засновників (вносили кошти)
            list_benef = list(set([person[0].lower().replace('  ', ' ') for person in firm['raw_persons'] 
                                   if person[1] == 'Бенефіціарний власник'])) # список бенефіціарів
            for head in list_master: # додаємо інформацію про голів фірм, вони можуть не бути засновниками
                list_head.append([head, edrpou])
            records = firm['raw_records'] # тут міститься інформація про частки участі засновників (у грн.)
            part = dict()
            for record in records: # відбираємо інформацію тільки про ненульові внески
                if (record.find(' грн.')!=-1) and (record.find(' 0.00 грн.')==-1) and (record.find(' 0,00 грн.')==-1):
                    part[record.split(',')[0].lower().replace('  ', ' ')] = float(
                        record.split('-')[-1].split('грн')[0].strip().replace(',', '.')) # засновник і його внесок
            if len(part) > 0: # якщо є інформація хоча б про один ненульовий внесок до статутного капіталу
                capital = sum(part.values())
                for founder in list(part):
                    if founder in list_all: # засновник -- фізична особа
                        if founder in list_benef: # засновник є бенефіціаром
                            list_type.append([founder, edrpou, part[founder]/capital, 1])
                        else: # засновник не є бенефіціаром
                            list_type.append([founder, edrpou, part[founder]/capital, 0])
                    else: # засновник -- юридична особа (фірма)
                        if founder in list(dict_firm): # і для цієї фірми ми вже маємо інформації про її ЄДРПОУ
                            list_type.append([dict_firm[founder], edrpou, part[founder]/capital, 0])
                        else: # інакше записуємо просто назву фірми
                            list_type.append([founder, edrpou, part[founder]/capital, 0])
            # якщо бенефіціар не вносив кошти до статутного капіталу, додаємо інформацію про нього
            zero_benef = list(set(list_benef) - set(list(part)))
            if len(zero_benef) > 0:
                for founder in zero_benef:
                    list_type.append([founder, edrpou, 0, 1])
            # якщо засновник не є бенефіціаром і не вносив кошти до статутного капіталу, додаємо інформацію про нього
            zero_founder = list(set(list_founder) - set(list_benef) - set(list(part)))
            if len(zero_founder) > 0:
                for founder in zero_founder:
                    list_type.append([founder, edrpou, 0, 0])

In [26]:
dict_find = dict() # словник знайдених контрагентів
dict_firm = dict() # словник назва фірми --> ЄДРПОУ
list_type = [] # список зв'язків між знайденими контрагентами
list_head = [] # список голів фірм
list_temp = [] # список контрагентів на поточному рівні
list_next = [] # список контрагентів наступного рівня
i = 0 # лічильник (номер рівня відносно початкового об'єкта)

In [28]:
# ці дані буде задавати користувач під час запуску
start = '35696771' # задаємо ЄДРПОУ початкової фірми
level = 3 # задаємо максимальну кількість рівнів

In [29]:
%%time
list_next.append((start, ))
while (len(list_next) > 0) and (i < level):
    list_temp = list_next.copy()
    list_next = []
    for obj in list_temp:
        resp = req(obj[0])
        num = resp[0] # кількість знайдених фірм, пов'язаних із даним об'єктом (ЄДРПОУ, назва фірми, П.І.Б.)
        if num != 0:
            list_obj = resp[1] # список даних по знайдених фірмах
            for n in range(num):
                edrpou = list_obj[n]['full_edrpou'] # підтягуємо ЄДРПОУ фірми (що робити, якщо його немає?)
                if edrpou not in set(dict_find.keys()): # якщо ми ще не обробляли інформацію про дану фірму
                    if len(obj) == 1:
                        add_info(list_obj[n]) # додаємо інформацію про засновників фірми та нові об'єкти для пошуку
                    else:
                        add_info(list_obj[n], obj[1]) # додаємо те ж саме, тільки для фірм із певної області
        else:
            if req(obj)[1] == ['error']:
                print('Error request for object = ', obj)
    i += 1

CPU times: user 3.31 s, sys: 203 ms, total: 3.51 s
Wall time: 1min 17s


In [30]:
dict_find # знайдені контрагенти (вершини графа), для фірм додаємо повну назву, скорочену назву і стан

{'35696771': ('товариство з обмеженою відповідальністю "комплекс інвестмент менеджмент"',
  'тов "кім"',
  'зареєстровано'),
 'ловчин лев юрійович': (),
 'рутовський дмитро олегович': (),
 'соколенко олександр іванович': (),
 '37761449': ('товариство з обмеженою відповідальністю "інджи"',
  'тов "інджи"',
  'зареєстровано'),
 'кравченко валентин валерійович': (),
 'меламуд євген мойсейович': (),
 '40581613': ('товариство з обмеженою відповідальністю "камидана"',
  'тов "камидана"',
  'зареєстровано'),
 'рутовська світлана олександрівна': (),
 '39189177': ('товариство з обмеженою відповідальністю "сервіс уют"',
  'тов "сервіс уют"',
  'зареєстровано'),
 'корольов сергій петрович': (),
 '39119103': ('товариство з обмеженою відповідальністю "домера"',
  'тов "домера"',
  'зареєстровано'),
 'рутовський олег георгійович': (),
 '30542625': ('товариство з обмеженою відповідальністю "южспецмонтаж"',
  'тов "южспецмонтаж"',
  'зареєстровано'),
 'корольов петро васильович': (),
 'анісіфоров воло

In [31]:
dict_firm # словник для визначення ЄДРПОУ фірми за її назвою (сподіваюсь, дублів за повними назвами фірм немає)

{'товариство з обмеженою відповідальністю "комплекс інвестмент менеджмент"': '35696771',
 'товариство з обмеженою відповідальністю "інджи"': '37761449',
 'товариство з обмеженою відповідальністю "камидана"': '40581613',
 'товариство з обмеженою відповідальністю "сервіс уют"': '39189177',
 'товариство з обмеженою відповідальністю "домера"': '39119103',
 'товариство з обмеженою відповідальністю "южспецмонтаж"': '30542625',
 'дочірнє підприємство "юсм-уют"': '33138719',
 'товариство з обмеженою відповідальністю "стройбілдінг"': '39324857',
 'товариство з обмеженою відповідальністю "коляда софт"': '42498353',
 'товариство з обмеженою відповідальністю "комфорт буд одеса"': '41398320',
 'акціонерне товариство "південсільспецмонтаж" закритого типу': '05392335',
 'товариство з обмеженою відповідальністю "навігейт україна"': '39009435',
 'мале підприємство у вигляді товариства з обмеженою відповідальністю фірма "сокол" лтд': '13904523',
 'громадська організація "організація інвалідів "новий кро

In [32]:
list_type # (засновник, ЄДРПОУ заснованої фірми, частка участі засновника, чи є засновник бенефіціаром)

[['соколенко олександр іванович', '35696771', 0.34, 1],
 ['рутовський дмитро олегович', '35696771', 0.33, 0],
 ['ловчин лев юрійович', '35696771', 0.33, 0],
 ['меламуд євген мойсейович', '37761449', 0.45, 0],
 ['35696771', '37761449', 0.55, 0],
 ['рутовський дмитро олегович', '40581613', 0.2792307142272753, 0],
 ['рутовська світлана олександрівна', '40581613', 0.0837692142681826, 0],
 ['товариство з обмеженою відповідальністю "южспецмонтаж"',
  '40581613',
  0.6370000715045421,
  0],
 ['рутовський дмитро олегович', '39189177', 1.0, 1],
 ['товариство з обмеженою відповідальністю "южспецмонтаж"',
  '39119103',
  0.23026733809523808,
  0],
 ['рутовський дмитро олегович', '39119103', 0.6983040904761905, 1],
 ['рутовський олег георгійович', '39119103', 0.03571428571428571, 1],
 ['рутовська світлана олександрівна', '39119103', 0.03571428571428571, 1],
 ['рутовський дмитро олегович', '30542625', 0.7692307692307693, 1],
 ['рутовський олег георгійович', '30542625', 0.23076923076923078, 1],
 ['3

Якщо ЄДРПОУ по якихось фірмах не підтягнулося відразу, то заміняємо зараз назву фірми на ЄДРПОУ (заміняється не завжди):

In [33]:
%%time
list_type_new = list_type.copy()
for i in range(len(list_type_new)):
    if list_type_new[i][0] in list(dict_firm):
        list_type_new[i][0] = dict_firm[list_type_new[i][0]]
list_type_new

CPU times: user 354 µs, sys: 35 µs, total: 389 µs
Wall time: 401 µs


[['соколенко олександр іванович', '35696771', 0.34, 1],
 ['рутовський дмитро олегович', '35696771', 0.33, 0],
 ['ловчин лев юрійович', '35696771', 0.33, 0],
 ['меламуд євген мойсейович', '37761449', 0.45, 0],
 ['35696771', '37761449', 0.55, 0],
 ['рутовський дмитро олегович', '40581613', 0.2792307142272753, 0],
 ['рутовська світлана олександрівна', '40581613', 0.0837692142681826, 0],
 ['30542625', '40581613', 0.6370000715045421, 0],
 ['рутовський дмитро олегович', '39189177', 1.0, 1],
 ['30542625', '39119103', 0.23026733809523808, 0],
 ['рутовський дмитро олегович', '39119103', 0.6983040904761905, 1],
 ['рутовський олег георгійович', '39119103', 0.03571428571428571, 1],
 ['рутовська світлана олександрівна', '39119103', 0.03571428571428571, 1],
 ['рутовський дмитро олегович', '30542625', 0.7692307692307693, 1],
 ['рутовський олег георгійович', '30542625', 0.23076923076923078, 1],
 ['30542625', '33138719', 1.0, 0],
 ['рутовський олег георгійович', '33138719', 0, 1],
 ['рутовський дмитро 

In [23]:
list_head # (голова, ЄДРПОУ фірми)

[['соколенко олександр іванович', '35696771'],
 ['кравченко валентин валерійович', '37761449'],
 ['рутовський дмитро олегович', '40581613'],
 ['корольов сергій петрович', '39189177'],
 ['рутовський дмитро олегович', '39119103'],
 ['галак оксана василівна', '30542625'],
 ['малюта ірина михайлівна', '30542625'],
 ['корольов петро васильович', '30542625'],
 ['анісіфоров володимир львович', '30542625'],
 ['саіпова наталія леонідівна', '33138719'],
 ['жоров євген данилович', '39324857'],
 ['гук вячеслав олександрович', '42498353'],
 ['тростянецький володимир васильович', '42498353'],
 ['тростянецький володимир васильович', '41398320'],
 ['діланян самвел авакович', '41398320'],
 ['дорошенко анатолій іванович', '41398320'],
 ['анісіфоров володимир львович', '39009435'],
 ['нарольський андрій олегович', '39009435'],
 ['корольов петро васильович', '39009435']]

In [24]:
list_temp # список контрагентів, за якими здійснювався пошук на останньому рівні

[('товариство з обмеженою відповідальністю "комплекс інвестмент менеджмент"',),
 ('ловчин лев юрійович', 5),
 ('рутовський дмитро олегович', 5),
 ('соколенко олександр іванович', 5)]

In [25]:
list_next # список контрагентів для пошуку на наступному рівні (непорожній, якщо було задано максимальний рівень)

[('товариство з обмеженою відповідальністю "інджи"',),
 ('кравченко валентин валерійович', 5),
 ('меламуд євген мойсейович', 5),
 ('товариство з обмеженою відповідальністю "камидана"',),
 ('рутовська світлана олександрівна', 5),
 ('рутовський дмитро олегович', 5),
 ('товариство з обмеженою відповідальністю "сервіс уют"',),
 ('рутовський дмитро олегович', 5),
 ('корольов сергій петрович', 5),
 ('товариство з обмеженою відповідальністю "домера"',),
 ('рутовський олег георгійович', 5),
 ('рутовська світлана олександрівна', 5),
 ('рутовський дмитро олегович', 5),
 ('товариство з обмеженою відповідальністю "южспецмонтаж"',),
 ('корольов петро васильович', 5),
 ('рутовський дмитро олегович', 5),
 ('рутовський олег георгійович', 5),
 ('анісіфоров володимир львович', 5),
 ('галак оксана василівна', 5),
 ('малюта ірина михайлівна', 5),
 ('дочірнє підприємство "юсм-уют"',),
 ('рутовський олег георгійович', 5),
 ('саіпова наталія леонідівна', 5),
 ('рутовський дмитро олегович', 5),
 ('товариство 